<a href="https://colab.research.google.com/github/czig/afocd/blob/master/Classification_solve_max_targets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification Model Step to Solve for Max Targets

### Problem

In the past, we have seen that the model does not have enough cadets with the correct qualifications to satisfy the mandatory requirement for some AFSCs. The mandatory requirement says that AFSC X needs Y% of their accessions to have a degree from list Z. When the model cannot meet the mandatory requirement, it simply breaks, leading to time-consuming debugging to find out which AFSC's requirement broke the model. 

### Methodology

Since a cadet meeting the mandatory requirement for one AFSC probably meets the mandatory requirement for another AFSC, we cannot find this answer from a simple query. One way to solve this would be to run a "pre-solver" after preprocessing the data but before running the Classification model integer program. The "pre-solver" would be an integer program itself, but it would solve for the maximum possible targets given the pool of cadets and mandatory constraints _only_. 


#### Givens:

* $n$: number of cadets to be classified

* $m$: number of AFSCs that require accessions from model

* $Tier_M$ with dimensions $n \times m$, where each row is a cadet (indexed by ssan) and each column is an AFSC. For $1 \le i \le n$ and $1 \le j \le m$, $Tier_M[i,j]$ is a binary variable that takes a value of 1 if cadet $i$ meets the mandatory requirement for AFSC $j$ else 0. 

* $targets$ with dimensions $m \times 1$, where each row is an AFSC (indexed by AFSC). For $1 \le j \le m$, $targets[j]$ is the target provided by A1PT for AFSC $j$

* $Rate_M$ with dimensions $m \times 1$, where each row is an AFSC (indexed by AFSC). For $1 \le j \le m$, $Rate_M[j]$ is the percentage of accessions that must meet the mandatory requirement for AFSC $j$

* $supply_M$ with dimensions $m \times 1$, where each row is an AFSC (indexed by AFSC). For $1 \le j \le m$, $supply_M$ is the amount of cadets who are "mandatory-qualified" for AFSC $j$. 
> $supply_M[j] = \sum_{i=1}^{n} Tier_M[i,j] $ for $1 \le j \le m$

#### Formulation:

Our goal is to find out the maximum number of "mandatory-qualified" cadets we can place in each AFSC. A few assumptions:
* We are most worried about percentage fill. For a given AFSC, only accessing 1 cadet when target is 2 is much worse than only accessing 99 when target is 100.
* Each mandatory tier is created equal, meaning we will not prioritize one AFSC's mandatory tier over another

Let $x_{i,j}$ be our binary decision variable. Matrix $X$ is size $n \times m$ where $X[i,j] = x_{i,j}$

#### Objective Function:
We have two options with objective function, first one tries to maximize percent fill, and the second one tries to maximize placement in AFSCs with low mandatory supply.
> $max \sum_{i=1}^{n} \sum_{j=1}^{m} \dfrac{Tier_M[i,j]*X[i,j]}{targets[j]*Rate_M[j]} $

Above, we divide by the number of mandatory targets to maximize the percentage fill.

> $max \sum_{i=1}^{n} \sum_{j=1}^{m} \dfrac{Tier_M[i,j]*X[i,j]}{supply_M[j]} $

Above, we divide by the number of "mandatory-qualified" cadets for AFSC $j$ to maximize placement in AFSCs that do not have many cadets eligible (i.e. Weather, Electrical Engineering, Aeronautical Engineering).

**Either of these objective functions could work, but I think the new formulation, shown below in 'Formulation part 2' does a better job of distributing the cadets equitably**

#### Constraints:
1. Cadet cannot be assigned to more than one AFSC (zero is allowed): 

> $\sum_{j=1}^m X[i,j] \le 1$, for $1 \le i \le n$

2. Do not assign more than the target: 

> $\sum_{i=1}^n X[i,j] \le targets[j]*Rate_M[j]$, for $1 \le j \le m$

#### Outputs:
We will get a matrix of binary decision variables as output, and we need to convert those into new targets that we can feed into the main linear program. This looks like: 

> $targets_{model}[j] = \Big\lfloor\dfrac{\sum_{i=1}^n X[i,j]}{Rate_M[j]}\Big\rfloor$, for $1 \le j \le m$



#### Formulation, part 2

Keep the same assumptions and variables from above, but let's try to minimize the difference between the mandatory target and the number of cadets we can assign. We need an absolute value for this, and since absolute value is _only_ piecewise linear, we need to adjust the formulation to make this work in a linear program.

For simplicity, lets define a new variable that is just derived from two others.

> $ num_{mandatory}[j] = targets[j]*Rate_M[j] $ for $ 1 \le j \le m $

#### Objective Function

Let $Y$ be $ 1 \times m $ matrix of decision variables, $y_j$, where $y_j$ denotes the absolute difference in 'mandatory-qualified' cadets assigned an AFSC and the mandatory requirement

> $min \sum_{j=1}^{m} Y[j]  $

#### Implicit variable

SAS let's us do this thing called creating implicit variables, and basically, it lets us write up an equation that gets "stored" in a variable. However, I don't know much more than that.

> $ X_{eqn}[j] = \dfrac{\big(\sum_{i=1}^{n} Tier_M[i,j]*X[i,j]\big) - num_{mandatory}[j]}{num_{mandatory}[j]} $, for $ 1 \le j \le m $

#### Constraints

1. Absolute value constraint

> $ Y[j] \ge -X_{eqn}[j] $, for $ 1 \le j \le m $

2. Absolute value constraint #2

> $ Y[j] \ge X_{eqn}[j] $, for $ 1 \le j \le m $

3. Force $x_{ij}$ to be binary decision variable: 

> $ X[i,j] \in \{0,1\} $, for $1 \le i \le n, 1 \le j \le m$

4. Cadet cannot be assigned to more than one AFSC (zero is allowed): 

> $\sum_{j=1}^m X[i,j] \le 1$, for $1 \le i \le n$









